In [5]:
from django.contrib.gis.geos import LineString, Polygon, MultiPolygon, GEOSGeometry, LinearRing
import json

# Load the GeoJSON file
geojson_file_path = '/home/teodorrk/projects/sql_chatbot/frontend/public/modified_map.geojson'
with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

new_features = []

for feature in geojson_data['features']:
    geometry = GEOSGeometry(json.dumps(feature['geometry']))

    # If the geometry is a LineString, try to convert it to Polygon
    if isinstance(geometry, LineString):
        # Ensure the LineString is closed
        if geometry[0] != geometry[-1]:
            closed_coords = list(geometry.coords) + [geometry.coords[0]]
            geometry = LineString(closed_coords)

        Linear_ring = LinearRing(geometry.coords)
        
        # Convert to Polygon
        polygon = Polygon(Linear_ring)
        feature['geometry'] = json.loads(polygon.geojson)
        feature['geometry']['type'] = 'Polygon'
    
    # Append updated feature
    new_features.append(feature)

# Update the GeoJSON data with the new features
geojson_data['features'] = new_features

# Save the modified GeoJSON back to a file
output_geojson_file_path = '/home/teodorrk/projects/sql_chatbot/frontend/public/converted_map.geojson'
with open(output_geojson_file_path, 'w') as f:
    json.dump(geojson_data, f, indent=2)

print(f"GeoJSON has been successfully converted and saved to {output_geojson_file_path}")

GeoJSON has been successfully converted and saved to /home/teodorrk/projects/sql_chatbot/frontend/public/converted_map.geojson


In [3]:
import json
from shapely.geometry import shape, Polygon
from shapely.ops import unary_union

# Load the GeoJSON file
geojson_file_path = '/home/teodorrk/projects/sql_chatbot/frontend/public/converted_map.geojson'
with open(geojson_file_path, 'r') as f:
    geojson_data = json.load(f)

# Step 1: Identify the outer border polygon (assuming it's the first feature)
border_polygon = shape(geojson_data['features'][0]['geometry'])

# Step 2: Initialize the remaining area to be the entire outer border
remaining_area = border_polygon

# New features list starting with the outer border
new_features = [geojson_data['features'][0]]  # Keep the border polygon unchanged

for feature in geojson_data['features'][1:]:
    # Get the shape of the current region
    region_polygon = shape(feature['geometry'])

    # Step 3: Check if the geometry is valid, if not fix it
    if not region_polygon.is_valid:
        region_polygon = region_polygon.buffer(0)  # Fix invalid geometry by cleaning

    # Check if the remaining area is valid
    if not remaining_area.is_valid:
        remaining_area = remaining_area.buffer(0)

    # Step 4: Clip the region with the remaining area (to avoid overlap)
    try:
        clipped_polygon = region_polygon.intersection(remaining_area)
    except Exception as e:
        print(f"Failed to clip region due to error: {e}")
        continue

    # Update the feature's geometry with the clipped polygon
    feature['geometry'] = clipped_polygon.__geo_interface__

    # Subtract this region from the remaining area
    remaining_area = remaining_area.difference(clipped_polygon)

    # Append the updated feature
    new_features.append(feature)

# Step 4: Save the modified GeoJSON back to a file
geojson_data['features'] = new_features
output_geojson_file_path = '/home/teodorrk/projects/sql_chatbot/frontend/public/aligned_non_overlapping_map.geojson'
with open(output_geojson_file_path, 'w') as f:
    json.dump(geojson_data, f, indent=2)

print(f"GeoJSON with non-overlapping regions has been saved to {output_geojson_file_path}")

GeoJSON with non-overlapping regions has been saved to /home/teodorrk/projects/sql_chatbot/frontend/public/aligned_non_overlapping_map.geojson
